# Trabalho 3 - Machine Learning II - Deep Learning
Prof: Carlos Padilha

#### Alunos:  

Roberto A. Coutinho  
Thais Galho

**Enunciado**

1.  Introdução a Deep Learning
    Este trabalho visa avaliar o entendimento em relação as redes convolutivas e recorrentes vistas em sala para resolver diversos problemas.  
    
    
2.  A proposta é que a dupla escolha alguma aplicação que mais se interessa e resolva usando os conceitos vistos em sala. Segue alguns exemplos de problemas que vocês podem escolher:

    * Classificação de imagens
    * Geração automática de imagens • Reconstrução de imagens
    * Análise de sentimento
    * Sumarização de textos
    * Predição em séries temporais

Obs. Importante lembrar que é desejado que vocês escolham bases de dados diferentes daquelas usadas em sala.
O trabalho pode ser feito em dupla e deve ser enviado por email (carlos.engcomp@gmail.com).

**Descrição do trabalho**

    1. Decidimos utilizar redes convolutivas para classificação multi-class de imagens de flores.
    2. Como otimizador usamos o Gradiente Descente Estocrático
    3. Flores: Tulipas, Dentes de Leão, Rosas, Girassóis e Margaridas
    4. Usamos o Dataset e ambiente de treinamento no Kaggle.
        * GPU on
        * Tensorflow + Keras


## Imports

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from PIL import Image
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend

from keras.optimizers import *
from keras.metrics import *
from keras.callbacks import *

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

script_dir = os.path.dirname(".")
training_set_path = os.path.join(script_dir, '../input/flowers/flowers/')
test_set_path = os.path.join(script_dir, '../input/flowers/flowers/')

# Any results you write to the current directory are saved as output.

ModuleNotFoundError: No module named 'keras'

## Leitura dos datasets de treinamento e teste

In [ ]:
batch_size = 32
input_size = (256, 256)

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255, 
                                  validation_split=0.33)

training_set = train_datagen.flow_from_directory(training_set_path,
                                                 target_size=input_size,
                                                 batch_size=batch_size,
                                                 subset="training",
                                                 class_mode='categorical')



test_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            subset="validation",
                                            class_mode='categorical',
                                            shuffle=False)

# Definição das camadas do modelo

In [ ]:
classifier = Sequential()

classifier.add(Conv2D(32, 
                      (3, 3), 
                      input_shape=(256,256,3), 
                      activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2, 2), 
                            data_format="channels_first"))

#Second
classifier.add(Conv2D(32, 
                      (3, 3), 
                      activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2, 2)))

#Third
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

#Flattening
classifier.add(Flatten())

# Full connection

classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=5, activation='softmax'))

### Definição da função de callback

In [ ]:
class LossHistory(Callback):
    def __init__(self):
        super().__init__()
        self.epoch_id = 0
        self.losses = ''

    def on_epoch_end(self, epoch, logs={}):
        self.losses += "Epoch {}: accuracy -> {:.4f}, val_accuracy -> {:.4f}\n"\
            .format(str(self.epoch_id), logs.get('acc'), logs.get('val_acc'))
        self.epoch_id += 1

    def on_train_begin(self, logs={}):
        self.losses += 'Iniciando treinamento...\n'
        
history = LossHistory()

## Compilação do modelo

In [ ]:
#opt = Adam(lr=1e-3, decay=1e-6)
opt = SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)

classifier.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

## Execução do treinamento

In [ ]:
classifier.fit_generator(training_set,
                         steps_per_epoch=1000/batch_size,
                         epochs=90,
                         validation_data=test_set,
                         validation_steps=100/batch_size,
                         workers=24,
                         callbacks=[history])

# Apresentação dos Resultados

  
  
Nessa seção são apresentados os resultados da execução do algoritmo AdaBooting, bem como observações gerais feitas ao longo do processo de desenvolvimento, experimentos e simulações.
A primeira lista são os resultados com a base de treinamento e a segunda lista resultados considerando a base de testes.

In [15]:
score = classifier.evaluate_generator(test_set, 45)
print(score)

[0.7678724118832791, 0.7212078651685393]


In [16]:
from sklearn.metrics import classification_report, confusion_matrix

validation_set = test_datagen.flow_from_directory(test_set_path,
                                            target_size=input_size,
                                            batch_size=batch_size,
                                            subset="validation",
                                            class_mode='categorical',
                                            shuffle=False)


#Confution Matrix and Classification Report
Y_pred = classifier.predict_generator(validation_set, 45)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_set.classes,y_pred))

Found 1424 images belonging to 5 classes.
Confusion Matrix
[[178  43  13  10   9]
 [ 10 297   5  30   5]
 [ 11  31 107   6 103]
 [  8  18   1 207   8]
 [ 11  28  26  21 238]]


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(validation_set.classes, y_pred)

0.7212078651685393

# Conclusões Finais

Observando através de diversas simulações, podemos ver pelo comportamento do erro de aprendizagem dos modelos na medida que vai sendo minimizado o resultado da acurácia, precisão e recall tende a ambos se equilibrarem de forma a maximizar o desempenho entre eles. 
